In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf

import sys
sys.path.append('/mnt/c/Users/kheut/code/covid19-forecasting/tf_model_1p5/')

from enum import Enum

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense

import tensorflow_probability as tfp
from scipy.stats import beta, truncnorm


# Local imports from model.py, data.py
from model import CovidModel, LogPoissonProb, get_logging_callbacks, Comp, Vax
from data import read_data, create_warmup
#from plots import make_all_plots

2021-10-18 12:27:07.444176: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-18 12:27:07.444206: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
transition_window =3

warmup_start = '20210428'
warmup_end = '20210430'
train_start = '20210501'
train_end = '20210731'
test_start = '20210801'
test_end = '20210831'

state = 'Massachusetts'
state_abbrev = 'MA'

data_dir = '../data'
covid_estim_date = '20210901'
hhs_date = '20210903'
owid_date = '20210903'

log_dir = './logs/new_warmup'

In [3]:
df = read_data(data_dir=data_dir,
               covid_estim_date=covid_estim_date,
               hhs_date=hhs_date,
               owid_date=owid_date,
               state=state, state_abbrev=state_abbrev)

In [4]:
class Comp(Enum):
    A = 0
    M = 1
    #X = 2
    #G = 3
    
class Vax(Enum):
    total = -1
    no = 0
    yes = 1

In [5]:
# get warmup arrays, splitting on vaccination status
warmup_asymp, warmup_mild, warmup_extreme = create_warmup(df, 
                                                          warmup_start, 
                                                          warmup_end,
                                                          0,0,0)

# re-combine
warmup_asymp = warmup_asymp[Vax.no.value] + warmup_asymp[Vax.yes.value]

In [6]:
delta = {}
delta[Vax.total.value] = {}
delta[Vax.total.value]['prior'] = {'a': 1.05, 'b': 20}
# delta is 0-1, so we will sigmoid the output of the variational posterior
# Here we use a reverse sigmoid to convert from a mean of 0.1
# I cant figure out the math to do for the scale parameter
delta[Vax.total.value]['posterior_init'] = {'loc': np.log(0.1/(1-0.1)), 'scale':0.1}

T_serial = {}
T_serial[Vax.total.value] = {}
T_serial[Vax.total.value]['prior'] ={'loc':5.8, 'scale':1}
# T_serial is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 5.8
# I cant figure out the math to do for the scale parameter
T_serial[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(5.8),
                                         'scale':0.1}


rho_M = {}
rho_M[Vax.total.value] = {}
rho_M[Vax.total.value]['prior'] = {'a': 31.8, 'b': 10.3}
# rho is 0-1, so we will sigmoid the output of the variational posterior
# Here we use a reverse sigmoid to convert from a mean of 0.76
# I cant figure out the math to do for the scale parameter
rho_M[Vax.total.value]['posterior_init'] = {'loc': np.log(0.76/(1-0.76)),
                                      'scale':0.1}

lambda_M = {}
lambda_M[Vax.total.value] = {}
lambda_M[Vax.total.value]['prior'] = {'loc': 4.7, 'scale': 1}
# lambda is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 4.7
# I cant figure out the math to do for the scale parameter
lambda_M[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(4.7),
                                         'scale':1}

nu_M = {}
nu_M[Vax.total.value] = {}
nu_M[Vax.total.value]['prior'] = {'loc': 3.1, 'scale': 1.2}
# nu is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 3.1
# I cant figure out the math to do for the scale parameter
nu_M[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(3.1),
                                     'scale':1.2}
delta = {}
delta[Vax.total.value] = {}
delta[Vax.total.value]['prior'] = {'a': 1.05, 'b': 20}
# delta is 0-1, so we will sigmoid the output of the variational posterior
# Here we use a reverse sigmoid to convert from a mean of 0.1
# I cant figure out the math to do for the scale parameter
delta[Vax.total.value]['posterior_init'] = {'loc': np.log(0.1/(1-0.1)), 'scale':0.1}

T_serial = {}
T_serial[Vax.total.value] = {}
T_serial[Vax.total.value]['prior'] ={'loc':5.8, 'scale':1}
# T_serial is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 5.8
# I cant figure out the math to do for the scale parameter
T_serial[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(5.8),
                                         'scale':0.1}


rho_M = {}
rho_M[Vax.total.value] = {}
rho_M[Vax.total.value]['prior'] = {'a': 31.8, 'b': 10.3}
# rho is 0-1, so we will sigmoid the output of the variational posterior
# Here we use a reverse sigmoid to convert from a mean of 0.76
# I cant figure out the math to do for the scale parameter
rho_M[Vax.total.value]['posterior_init'] = {'loc': np.log(0.76/(1-0.76)),
                                      'scale':0.1}

lambda_M = {}
lambda_M[Vax.total.value] = {}
lambda_M[Vax.total.value]['prior'] = {'loc': 4.7, 'scale': 1}
# lambda is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 4.7
# I cant figure out the math to do for the scale parameter
lambda_M[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(4.7),
                                         'scale':1}

nu_M = {}
nu_M[Vax.total.value] = {}
nu_M[Vax.total.value]['prior'] = {'loc': 3.1, 'scale': 1.2}
# nu is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 3.1
# I cant figure out the math to do for the scale parameter
nu_M[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(3.1),
                                     'scale':1.2}


2021-10-18 12:27:10.332331: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-10-18 12:27:10.332493: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-18 12:27:10.332541: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-10-18 12:27:10.332575: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-10-18 12:27:10.332605: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [7]:
warmup_A_params = {}
warmup_A_params[Vax.total.value] = {}
warmup_A_params[Vax.total.value]['prior'] = []
warmup_A_params[Vax.total.value]['posterior_init'] = []

for day in range(transition_window):
    warmup_A_params[Vax.total.value]['prior'].append({'loc': warmup_asymp[day],
                                                'scale': warmup_asymp[day]/10})
    # must be positive so reverse softplus the mean
    warmup_A_params[Vax.total.value]['posterior_init'].append({'loc': tf.cast(tfp.math.softplus_inverse(warmup_asymp[day]),dtype=tf.float32),
                                                         'scale': tf.cast(tfp.math.softplus_inverse(warmup_asymp[day]/10),dtype=tf.float32)})

In [8]:
tfp.math.softplus_inverse(warmup_asymp[day]/10)

<tf.Tensor: shape=(), dtype=float64, numpy=251.8626798999444>

In [12]:
model = CovidModel([Vax.total], [Comp.A, Comp.M],
                 transition_window,
                 delta, T_serial, rho_M, lambda_M, nu_M,
                 warmup_A_params, posterior_samples=1000)
loss = LogPoissonProb() 

In [18]:
x_train = tf.cast(df.loc[train_start:train_end,'Rt'].values, dtype=tf.float32)
y_test = tf.cast(df.loc[train_start:train_end,'mild'], dtype=tf.float32)

with tf.GradientTape() as tape:
    result = model.call(x_train)
    loss_val = loss(y_test, result.stack())
    loss_val += sum(model.losses)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/kheuton/anaconda3/envs/covid_vax_model/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3461, in run_code
    return outflag  File "/tmp/ipykernel_243/3644646644.py", line 7, in <module>
    loss_val += sum(model.losses)  File "/mnt/c/Users/kheut/code/covid19-forecasting/tf_model_1p5/model.py", line 128, in call
    return result  File "/home/kheuton/anaconda3/envs/covid_vax_model/lib/python3.7/site-packages/tensorflow/python/util/tf_should_use.py", line 249, in wrapped
    error_in_function=error_in_function)


In [19]:
loss_val

<tf.Tensor: shape=(), dtype=float32, numpy=97185.14>

In [20]:
grads = tape.gradient(loss_val, model.trainable_weights)

In [21]:
grads

[<tf.Tensor: shape=(), dtype=float32, numpy=-2.225037>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.17275281>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.014031086>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.37425444>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-17.685799>,
 <tf.Tensor: shape=(), dtype=float32, numpy=5.822873>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.049308077>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.8890647>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.7639199>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.6217932>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.004154929>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0001754643>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.008513215>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.0001009628>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.015218299>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.011818142>]

In [17]:
model.trainable_weights

[<tf.Variable 'delta_A_loc_-1:0' shape=() dtype=float32, numpy=-2.1972246>,
 <tf.Variable 'delta_A_scale_-1:0' shape=() dtype=float32, numpy=0.1>,
 <tf.Variable 'T_serial_A_loc_-1:0' shape=() dtype=float32, numpy=5.796968>,
 <tf.Variable 'T_serial_A_scale_-1:0' shape=() dtype=float32, numpy=0.1>,
 <tf.Variable 'rho_M_loc_-1:0' shape=() dtype=float32, numpy=1.1526796>,
 <tf.Variable 'rho_M_scale_-1:0' shape=() dtype=float32, numpy=0.1>,
 <tf.Variable 'lambda_M_loc_-1:0' shape=() dtype=float32, numpy=4.690863>,
 <tf.Variable 'lambda_M_scale_-1:0' shape=() dtype=float32, numpy=1.0>,
 <tf.Variable 'nu_M_loc_-1:0' shape=() dtype=float32, numpy=3.0539045>,
 <tf.Variable 'nu_M_scale_-1:0' shape=() dtype=float32, numpy=1.2>,
 <tf.Variable 'warmup_A_loc_-1:0' shape=() dtype=float32, numpy=2518.6091>,
 <tf.Variable 'warmup_A_scale_-1:0' shape=() dtype=float32, numpy=251.86093>,
 <tf.Variable 'warmup_A_loc_-1:0' shape=() dtype=float32, numpy=2520.2236>,
 <tf.Variable 'warmup_A_scale_-1:0' shape=(

In [13]:
pi_M_samples = tf.TensorArray(tf.float32, size=transition_window, clear_after_read=False,
                                           name='pi_M_samples')

for j in range(transition_window):
    poisson_M_samples = np.array([dist.log_prob(j+1) for dist in poisson_M_dist_samples])
    pi_M_samples = pi_M_samples.write(j, np.array([dist.log_prob(j + 1) for dist in poisson_M_dist_samples]) /
                                                   model.nu_M_samples_constrained)
pi_M_samples = pi_M_samples.stack()

In [14]:
poisson_M_dist_samples[0].log_prob(3)

<tf.Tensor: shape=(), dtype=float32, numpy=-1.5153024>

In [18]:
df.loc[train_start:train_end,'mild'].shape

(92,)

In [22]:
calc_poisson

NameError: name 'calc_poisson' is not defined

In [23]:
def calc_poisson(inputs):
    true_rate, predicted_rate = inputs
    poisson = tfp.distributions.Poisson(rate=true_rate)
    return poisson.log_prob(predicted_rate)

In [25]:
tf.map_fn(calc_poisson, (tf.squeeze(tf.cast(df.loc[train_start:test_end,'mild'], dtype=tf.float32)),
                         result.stack()), fn_output_signature=tf.float32)

<tf.Tensor: shape=(123, 1000), dtype=float32, numpy=
array([[-1833.6412, -1865.2865, -1850.6737, ..., -1845.1101, -1837.7191,
        -1865.5847],
       [-1843.8658, -1865.4167, -1855.9562, ..., -1851.882 , -1849.3024,
        -1865.7461],
       [-1858.0245, -1870.422 , -1866.347 , ..., -1862.8467, -1862.2065,
        -1869.534 ],
       ...,
       [-3063.9116, -3063.9116, -3063.9116, ..., -3063.9116, -3063.9116,
        -3063.9116],
       [-3162.4792, -3162.4792, -3162.4792, ..., -3162.4792, -3162.4792,
        -3162.4792],
       [-3266.1238, -3266.1238, -3266.1238, ..., -3266.1238, -3266.1238,
        -3266.1238]], dtype=float32)>